The following cells implement modules to train a convolutional neural network with binary kernels (for resources-constrained platforms). The final model is available in fourth cell. This software is companion to paper: 
R. Dogaru and Ioana Dogaru, "BCONV-ELM: Binary Weights Convolutional 
Neural Network Simulator based on Keras/Tensorflow for Low Complexity Implementations", Proceedings of ISEEE 2019, in Press. 

Please cite the above paper if you find this code useful 
- First module (BCONV) implements a 2-layer convolution expansion filter and applies it on a selected dataset providing the output tensors out_train and out_test which can be further used as inputs in additional layers (a linear adaptive unit or MLP0 is recommended for low complexity implements) 
- Second module (ELM)implements an Extreme Learning Machine and it is a revised version using Keras backend operators (thus supporting GPU) of the simulator available in https://github.com/radu-dogaru/ELM-super-fast  With its  0-neurons (no hidden layer) options it is used to optimize the randomly generated binary convolution kernels in BCONV for best performance. 
- Third module (MLP) implements a Keras multi-layer perceptron as output stage of the network (MLP0 - i.e. no hidden layer) is recommended for compact models. 
- Fourth module (QUANT) ensures the 8 bit fixed point quantization of the MLP0 weights and concludes the model as required by a resources-constrained implementation. 

Copyright Radu & Ioana DOGARU - radu_d@ieee.org , Aug. 13 2019  
License: https://choosealicense.com/licenses/bsd-3-clause/
Notice: On Kaggle platform please check to have Internet = ON GPU = ON (CPU works, but slowly)

In [ ]:
'''
BCONV MODULE 
This module loads dataset (one of MNIST, CIFAR10 or Fashion-MNIST)
# Copyright Radu & Ioana DOGARU - radu_d@ieee.org , Aug. 13 2019 
# More details in paper 
# R. Dogaru and Ioana Dogaru, "BCONV-ELM: Binary Weights Convolutional 
# Neural Network Simulator based on Keras/Tensorflow for Low Complexity 
# Implementations", in Proceedings ISEEE 2019, in Press. 
# Please cite the above paper if you find this code useful 
'''
import keras.backend as K
import tensorflow as tf
import scipy.linalg as sclin
import numpy as np
import time as ti
from sklearn.preprocessing import StandardScaler
'''
Uses K.backend methods to implement (GPU/CPU) a preprocessing 
layer with up to 2 convolutional layers (each with abs() 
value nonlinearity and pooling)
Convolution kernels are BINARY and RANDOM !
THIS MODULE WOULD GENERATE out_train out_test 
AS THE OUTPUT OF A PREPROCESSING NONLINEAR LAYER 
WITH UP TO 2 CONVOLUTIONAL LAYERS AND x_train x_test tuple at the input. 
'''

#------------------------ Algo parameters ----------
########################################################
dataset='mnist' # or f-mnist or cifar10 
# convolutional layers 
typ_con=1 #  1 = depthwise (); 2 = normal (as usually used in Keras CNNs)
filtre1=10; filtre2=16 # if filtre2=0 only the first layer is implemented  
ksize1=3; ksize2=3    # kernel size on layers 1 & 2 
pool=(2,2); pool2=(3,3)  # pool size on layers 1 & 2 
strp1=2; strp2=2 # strides on pooling layers 1 & 2 
strc1=1; strc2=1 # strides on conv layers 1 & 2 
pmode='max'  # pooling mode (alternative is  'average', lower performance) 
pad='same' # 'same' (alternative is  'valid' ) results in less outputs but performance may decrease
reduced = 10000 # 0-full; >0 = reduced set of training samples (using all samples - choose 0)
use_stored = 0 # use bk wher bk=(ker,ker2) are best kernels (saved after run in console mode)
# choose 0 above for loop tuning (then run all cells 5-10 trials saving bk=(ker,ker2) for any improvement in ELM0 accuracy)
# choose 1 above after tuning then run all cells (now ELM is not active)
epoci=1 # epochs in the MLP (use 1 in the ELM loop tuning phase with Run ALL)

print('Dataset is:', dataset)
print('Convolution type is (1-depthwise, 2-classic)',typ_con)
print('Layer 1: filters:',filtre1,' kernel size:',ksize1,'pooling size:',pool,'strides pooling:',strp1)
print('Layer 1: filters:',filtre2,' kernel size:',ksize2,'pooling size:',pool2,'strides pooling:',strp2)
print('Pooling mode is:',pmode,'  Padding is: ',pad)

# definitie primul strat de convolutii (liniare)
def convlayer(inlay,ker_,pool,strp,strc,typ_con,pad,pmode):
# inlay este intrarea (multicanal si organizata [samples, x_size, y_size, channells])
# ker_ este variabila kernel organizata in forma [x_size, x_sixe, channels, filtre ]
    dformat='channels_last'
    if typ_con==1:
        out_=K.depthwise_conv2d(inlay, ker_, strides=(1, 1), padding=pad, data_format=dformat)
    elif typ_con==2:
        out_=K.conv2d(inlay, ker_, strides=(1, 1), padding=pad, data_format=dformat)
        
    nout_=K.pool2d(out_,pool, strides=(strp, strp), padding=pad, data_format=dformat ,pool_mode=pmode)
    nout__=K.abs(nout_) # useful nonlinearity   (instead of  RELU)
    out_=K.batch_flatten(nout__)  # flatten all ELM 
    return out_, nout_

# Datasets import (Internet must be ON)
# mnist, cifar have inputs integers up to 255 
from keras.datasets import mnist, cifar10, fashion_mnist

if dataset=='mnist':
    (x_train, y_train), (x_test, y_test) = mnist.load_data() # incarca date nescalate 
elif  dataset=='cifar10': 
    (x_train, y_train), (x_test, y_test) = cifar10.load_data() # incarca date nescalate 
elif dataset=='f-mnist':
    (x_train, y_train), (x_test, y_test) =  fashion_mnist.load_data()

if (np.ndim(x_train)==3):   # E.g.  MNIST or F-MNIST  
    x_train=np.reshape(x_train, [np.shape(x_train)[0],np.shape(x_train)[1],np.shape(x_train)[2], 1]) 
    x_test=np.reshape(x_test, [np.shape(x_test)[0],np.shape(x_test)[1],np.shape(x_test)[2], 1] ) 
# place a  1 in the end to keep it compatible with kernel in conv2d 
# scaling in ([0,1])
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /=255 

# one can choose a lower numbers of training samples (when GPU MEM is overloaded)
if reduced>0:
    Ntr1=reduced
    x_train=x_train[0:Ntr1,:,:,:]
    y_train=y_train[0:Ntr1]

#================ Apply the conv. layers ==============================
inp_chan=np.shape(x_train)[3] 
print('Number of input channels in image:', inp_chan)

# Construct model (binary random kernels) 
# Layer 1 kernels (without bias)
ker=np.sign(np.random.rand(ksize1,ksize1,inp_chan,filtre1).astype('float32')-0.5)
#ker=(np.random.rand(ksize1,ksize1,inp_chan,filtre1).astype('float32')-0.5)
ker=1*(ker)/(ksize1*ksize1) #scaling 

if use_stored==1: 
    ker=bk[0]
# Layer 2 kernels (without bias)
if typ_con==1:
    ker2=np.sign(np.random.rand(ksize2,ksize2,filtre1*inp_chan,filtre2).astype('float32')-0.5)
    #ker2=np.random.rand(ksize2,ksize2,filtre1*inp_chan,filtre2).astype('float32')-0.5

elif typ_con==2: 
    ker2=np.sign(np.random.rand(ksize2,ksize2,filtre1,filtre2).astype('float32')-0.5)
    # scaling 
    #ker2=1*(ker2)/(ksize2*ksize2)
    #ker2=bker2
    #There are 4 dimensions  (a,b,c,d) a,b dim. kernel ; 
    # c -numeber of channels (1 or 3 for the input, more for 2'nd layer) 
    # d is the number of  filters opened by this convolution layer .. 
if use_stored==1:
    ker2=bk[1]    
# Implementing the processing flow for (train / test) 
intrain=K.variable(x_train)
intest=K.variable(x_test)
ker_=K.variable(ker)
ker2_=K.variable(ker2)

out_train1_, nout_tr=convlayer(intrain,ker_,pool,strp1,strc1,typ_con,pad,pmode)
out_test1_, nout_ts=convlayer(intest,ker_,pool,strp1,strc1,typ_con,pad,pmode)
# Note: out_ are "flatten" outputs,  nout_ are structured outputs  
print('Layer 1:',np.shape(nout_tr))
# Plus an additional conv layer  
if filtre2>0: 
    out_train_, nout_tr2=convlayer(nout_tr,ker2_,pool2,strp2,strc2,typ_con,pad,pmode)
    out_test_, nout_ts2=convlayer(nout_ts,ker2_,pool2,strp2,strc2,typ_con,pad,pmode) 
    print('Layer 2:',np.shape(nout_tr2))
else:
    out_train_=out_train1_
    out_test_=out_test1_
    
# Effective computation of the input preprocessing flow 
t1=ti.time()
out_train=K.eval(out_train_)
out_test=K.eval(out_test_)
t2=ti.time()
K.clear_session()  # to avoid overloads 

# Number of parameters 


print('Convolutional layers processing time (train + test): ',t2-t1)
print('Number of bits in KER1:',np.size(ker))
print('Number of bits in KER2:',np.size(ker2))
print('Output structure:  ',np.shape(out_train))


In [ ]:
'''
#  ELM MODULE  
#================================================================================
#  Due to limited GPU RAM - input size should be smaller than 5000 on Kaggle 
#  or arrange to have less input samples (e.g. 10000 input samples)
#  On other platforms it depends on the available resources. 
#  For larger inputs size is better to use the latest cell (implementing a
#  typical multilayer perceptron  MLP in Keras)
# Copyright Radu & Ioana DOGARU - radu_d@ieee.org 
# More details in paper 
# R. Dogaru and Ioana Dogaru, "BCONV-ELM: Binary Weights Convolutional 
# Neural Network Simulator based on Keras/Tensorflow for Low Complexity 
# Implementations", in Proceedings ISEEE 2019, in Press. 
# Please cite the above paper if you find this code useful 
# 
#--------------------------------------------------------------------------
'''

nr_neuroni=0 # Proposed number of neurons on the hidden layer 
C=100 # Regularization coefficient C  (small value / useful for 0 neurons)
if nr_neuroni==0:
    C=0.1 # only for no-hidden layer 
tip=3 # Nonlinearity of the hidden layer (-1 means linear layer)
if nr_neuroni==0:
    tip=-1   # 
nb_in=2;  # 0 = float; x - represents weights on a finite x number of bits 
nb_out=8; # same as above but for the output layer
orig=0 # 0 - works with (out_train out_test) from BCONV; 
       # 1 works with original data - no conv. layers (x_train x_test)
       # 0 - convolved data ; 1 - directly applied to ELM 
#===============  TRAIN DATASET LOADING ==========================================

def hidden(x_,inw_,tip):
# Hidden layer definit ca "flow" Keras (argumentele sunt "variables")
      hin_=K.dot(inw_,x_)
      #----------  HIDDEN LAYER --------- 
      if tip==-1:  # liniar (Adaline only)
        h_=hin_
      elif tip==0: # tanh
        h_=K.tanh(hin_)
      elif tip==1:  # linsat 
        h_=K.abs(1+hin_)-K.abs(1-hin_)
      elif tip==2: # ReLU
        h_=K.relu(hin_)
      elif tip==3: 
            h_=K.abs(hin_)
      elif tip==4:
            h_=K.sqrt(K.square(hin_)+1)
      #------------------------------------ 
      return h_

# implements the ELM training procedure with weight quantization       
def elmTrain_fix( X, Y, h_Neurons, C , tip, ni):
# Training phase - emulated fixed point precision (ni bit quantization)
# X - Samples (feature vectors) Y - Labels
# ni - number of bits to quantize the inW weights 
      Ntr = np.size(X,1)
      in_Neurons = np.size(X,0)
      classes = np.max(Y)
      # transforms label into binary columns  
      targets = np.zeros( (classes, Ntr), dtype='int8' )
      for i in range(0,Ntr):
          targets[Y[i]-1, i ] = 1
      targets = targets * 2 - 1
      
      #   Generare inW 
      #   Generate inW layer 
      #   Takes care if h_Neurons==0 
      if h_Neurons==0:
          inW=np.eye(in_Neurons)
          h_Neurons=in_Neurons
    
      else: 
          rnd = np.random.RandomState()
          inW=-1+2*rnd.rand(h_Neurons, in_Neurons).astype('float32')
          #inW=rnd.randn(nHiddenNeurons, nInputNeurons).astype('float32')
          if ni>0:
            Qi=-1+pow(2,ni-1) 
            inW=np.round(inW*Qi)
      
      #  Compute hidden layer 
      iw_=K.variable(inW)
      x_=K.variable(X)
      h_=hidden(x_,iw_,tip)  
      #------------------------------------      
      # Moore - Penrose computation of output weights (outW) layer 
      ta_=K.variable(targets)
      print('KERAS ACTIVE')
      if h_Neurons<Ntr:
          print('LLL - Less neurons than training samples')
          outw_=tf.matrix_solve(K.eye(h_Neurons)/C+K.dot(h_,K.transpose(h_)),K.dot(h_,K.transpose(ta_)))  
      else:
          print('MMM - More neurons than training samples')
          outw_=K.dot(h_,tf.matrix_solve(K.eye(Ntr)/C+K.dot(K.transpose(h_),h_),K.transpose(ta_)))
      outW=K.eval(outw_)   
      K.clear_session()     
      return inW, outW 
      

def elmPredict_optim( X, inW, outW, tip):
# implements the ELM predictor given the model as arguments 
# model is simply given by inW, outW and tip 
# returns a score matrix (winner class has the maximal score)
      x_=K.variable(X)
      iw_=K.variable(inW)
      ow_=K.variable(outW)
      h_=hidden(x_,iw_,tip) 
      mul1=K.dot(K.transpose(h_),ow_)
      sc_=K.transpose(mul1)
      score = K.eval(sc_)
      K.clear_session() 
      return score 
        

#===============  TRAIN DATASET LOADING ==========================================
# converts out_train, y_train into Samples Labels 

if orig==1:
    intrain=K.variable(x_train)
    Samples_=K.batch_flatten(intrain)  # aici se aplica direct datele de intrare 
    Samples=(K.eval(Samples_)).T
else:
    Samples=out_train.T 

Labels=(y_train.T+1).astype('int8')
if (np.ndim(Labels)<2): 
    Labels=np.reshape(Labels,[1,np.shape(Labels)[0]])
clase=np.max(Labels)
#================= TRAIN ELM =====================================================
t1 = ti.time()
inW, outW = elmTrain_fix(Samples, np.transpose(Labels), nr_neuroni, C, tip, nb_in)
trun = ti.time()-t1
print(" training time: %f seconds" %trun)

# ==============  Quantify the output layer ======================================
Qout=-1+pow(2,nb_out-1)
if nb_out>0:
     O=np.max(np.abs(outW))
     outW=np.round(outW*(1/O)*Qout)

#================= TEST (VALIDATION) DATASET LOADING 


if orig==1:
    intest=K.variable(x_test)
    Samples_=K.batch_flatten(intest)  # aici se aplica direct datele de intrare 
    Samples=(K.eval(Samples_)).T
else: 
    Samples=out_test.T 

Labels=(y_test.T+1).astype('int8')
if (np.ndim(Labels)<2):
    Labels=np.reshape(Labels,[1,np.shape(Labels)[0]])  # acopera cazul MNIST 

n=Samples.shape[0]
N=Samples.shape[1]

#====================== VALIDATION PHASE (+ Accuracy evaluation) =================
t1 = ti.time()
scores = elmPredict_optim(Samples, inW, outW, tip)
trun = ti.time()-t1
print( " prediction time: %f seconds" %trun)

# CONFUSION MATRIX computation ==================================
Conf=np.zeros((clase,clase),dtype='int16')
for i in range(N):
    # gasire pozitie clasa prezisa 
    ix=np.where(scores[:,i]==np.max(scores[:,i]))
    ixx=np.array(ix)
    pred=int(ixx[0,0])
    actual=Labels[0,i]-1
    Conf[actual,pred]+=1
accuracy=100.0*np.sum(np.diag(Conf))/np.sum(np.sum(Conf))
print("Confusion matrix is: ")
print(Conf)
print("Accuracy is: %f" %accuracy)
print( "Number of hidden neurons: %d" %nr_neuroni)
print( "Hidden nonlinearity (0=sigmoid; 1=linsat; 2=Relu; 3 - ABS; 4- multiquadric): %d" %tip)
K.clear_session() 
#====================================================================================   


In [ ]:
# -----------------------------------------------------------------------
# KERAS MLP (Up to 2 hidden layer - 0 hidden layers (MLP0)= Adaline 
#  is recommended for low cpx.) 
#  Slower than  ELM0, but gives better accuracies
#  Fixed point quantization of the MLP0 is implemented in the last cell !
#--------------------------------------------------------
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop, SGD, Adadelta, Adam, Nadam 

import matplotlib.pyplot as plt

batch_size = 100  #  (small values - larger training times but better accuraccy ) 
#epoci = 100 # Number of epochs 
train_style = 2 # 1-the Keras usual in examples ; 2 keeps best result during training in all epochs (recommended)
#--------------  structura model neuronal --------------------------------------------------------
nescalat =0  # 0 for scaled data (1 additional scalling) 
nhid1 = 0 # hidden1 - take 0 for MLP0 (Adaline) - if > 0 nhid2 may also be used 
nhid2 = 0 # hidden2 - take 0 for MLP0 (or other value if nhid1>0)
drop1= 0.0 # droput if drop1>0
# Liniar output layer (nhid1=nhid2=0)
# ------------------- Optimizer -----------------------------------------------------------------
#myopt=SGD(lr=0.01)
#myopt = SGD(lr=.007, decay=1e-6, momentum=.9, nesterov=True)
myopt =Adadelta(lr=0.1)  # implicit are lr=1 # cum influenteaza valoarea procesul de antrenare ?? 
#myopt = RMSprop(lr=0.01) 
#myopt = Nadam()
#myopt=Adam()

# -------------------------- Loss choice ------------------------------------
#my_loss='mean_squared_error'  # desigur se pot alege si alte versiuni 
#my_loss='mean_absolute_error'
my_loss='categorical_crossentropy'

# ---------------  conversie  categorical (vector coloana cu 1 pe linia asociata clasei) 
num_classes = 1+np.max(y_test).astype('int8')
num_inputs = np.shape(out_test)[1]
print('Number of inputs: ',num_inputs)
yc_train = keras.utils.to_categorical(y_train, num_classes)
yc_test = keras.utils.to_categorical(y_test, num_classes)

#------------  MODEL Definition  ----------------------------------------
# Nota: se poate construi propriul model (vezi https://gist.github.com/abhaikollara/430c0491c851cf0b05a852f1faa805d7  )
# ----------------------------------------------
model = Sequential()
if nhid1>0:
    model.add(Dense(nhid1, activation='relu', input_shape=(num_inputs,)))
    if drop1>0: 
        model.add(Dropout(drop1))
# second hidden layer 
if nhid2>0:
    model.add(Dense(nhid2, activation='relu'))
#   model.add(Dropout(0.2))
# output layer 
if (nhid1+nhid2)==0:
    model.add(Dense(num_classes, activation='softmax',input_shape=(num_inputs,)))
else: 
    model.add(Dense(num_classes, activation='softmax'))


# --- MODEL COMPILE ---------------------------------------

model.compile(loss=my_loss, 
              optimizer=myopt,   
              metrics=['accuracy'])

# --- RUNS THE MODEL 
#----
model.summary()

err_test=np.zeros(epoci)   # For plot   
best_acc=0.0
best_ep=0
t1=ti.time()
for k in range(epoci):
    model.fit(out_train, yc_train,
              batch_size=batch_size,
              epochs=1,
              verbose=0,  #  0 (no details) 1 (details) 2(only epochs)
              validation_data=(out_test, yc_test))
    score = model.evaluate(out_test, yc_test, verbose=0)
    err_test[k]=score[1]
    print('Epoch:',k,' ACC:',100*score[1],'| ')
    if score[1]>best_acc : 
            print('improved in epoch:', k, 'best accuracy so far: ', 100*score[1],'%')
            best_acc=score[1]
            best_ep=k
            best_pars=model.get_weights()
t2=ti.time()
print('Best accuracy:', best_acc*100, '% obtained in epoch: ',best_ep, ' running  ',epoci,' epochs in ',t2-t1,' seconds')
t1=ti.time()
score = model.evaluate(out_test, yc_test, verbose=0); 
t2=ti.time()
print('Time for prediction (test set):', t2-t1)
plt.plot(err_test)
print('If quantization is desired see the next cell')

In [ ]:
# QMLP0 (quantified MLP0 module)
# Quantization of the above resulted model (only for MLP0)
# Copyright Radu and Ioana DOGARU;  radu.dogaru@ieee.org
#=============================================================
nb_out=8
outW=np.copy(best_pars)
Qout=-1+pow(2,nb_out-1)
if (nb_out >0) & (nhid1==0) :
    O=np.max(np.abs(outW[0]))
    outW[0]=np.round(outW[0]*(1/O)*Qout)
    outW[1]=np.round(outW[1]*(1/O)*Qout)
    model.set_weights(outW)
    score = model.evaluate(out_test, yc_test, verbose=0)
    best_acc=score[1]
    print('Output layer quantized with:', nb_out, 'bits')
    print('Quantified accuracy is:', best_acc*100,'%')
outW=model.get_weights() # the resulting model 
Bconvmodel=(ker,ker2,outW)